In [1]:
"""
Full SPEI_N classification pipeline (SVM, RF, MLP) with:
- Data from SQLite (table 'data')
- Parametric target: SPEI_1_Class, SPEI_3_Class, SPEI_6_Class, ...
- Preprocessing: imputation, scaling, one-hot encoding
- Class imbalance: SMOTE + class_weight
- Feature selection: SelectKBest (mutual_info_classif)
- Models: SVM, RandomForest, MLP
- Hyperparameter tuning: RandomizedSearchCV
- Evaluation: test metrics, per-class report, 5-fold CV
- Feature importance: permutation importance
- For each target, outputs are stored in its own folder under output_root (e.g. results/SPEI_6_Class/)
"""

import warnings
warnings.filterwarnings("ignore")

import os
import sqlite3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import (
    train_test_split,
    RandomizedSearchCV,
    StratifiedKFold,
    cross_validate
)
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from joblib import dump


RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)


# ---------------------------------------------------
# 1. Helpers: Class distribution plotting
# ---------------------------------------------------
def plot_class_distribution(y, title="Class distribution", save_as=None):
    """
    Print and plot class distribution (counts and ratios).
    """
    counts = y.value_counts().sort_index()
    ratios = counts / counts.sum()

    print(f"\n{title}")
    print(pd.DataFrame({"count": counts, "ratio": ratios}))

    plt.figure(figsize=(6, 4))
    sns.barplot(x=counts.index, y=counts.values)
    plt.ylabel("Count")
    plt.xlabel("Class")
    plt.title(title)
    plt.tight_layout()
    if save_as is not None:
        plt.savefig(save_as, dpi=300)
    plt.close()


# ---------------------------------------------------
# 2. Load Data from SQLite
# ---------------------------------------------------
def load_data_sqlite(
    db_path="../database/database.db",
    table_name="data",
    target_col=None,
    selected_columns=None,
    selected_stations=None,
    start_date=None,
    end_date=None,
):
    """
    Load data from a SQLite database, drop rows with missing target_col,
    and extract year/month from 'date' column if present.
    """   
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"Database file not found: {db_path}")

    conn = sqlite3.connect(db_path)
    try:
        data = pd.read_sql(sql=f"SELECT * FROM {table_name}", con=conn)
    finally:
        conn.close()

    
    df = data\
        .filter(items=selected_columns)\
            .query("station_name in @selected_stations and date >= @start_date and date < @end_date")
    
    if "_Class" in target_col:
        SPEI_LABELS = ['ED', 'SD', 'MD', 'NN', 'MW', 'VW', 'EW']
        SPEI_BINS = [-10, -2, -1.5, -1, 1, 1.5, 2, 10]
        df[f'{target_col}'] = pd.cut(df[target_col.replace("_Class", "")], bins=SPEI_BINS, labels=SPEI_LABELS)
        df[f'{target_col}'] = df[f'{target_col}'].astype('category')
        
        df[f'GPM_{target_col}'] = pd.cut(df[f'GPM_{target_col.replace("_Class", "")}'], bins=SPEI_BINS, labels=SPEI_LABELS)
        df[f'GPM_{target_col}'] = df[f'GPM_{target_col}'].astype('category')
    
    
    # Extract year/month from date if exists
    if "date" in df.columns:
        df['date'] = pd.to_datetime(df['date'], format='%Y-%m')
        df["year"] = df["date"].dt.year
        df["month"] = df["date"].dt.month
        df['month_sin'] = np.sin(2 * np.pi * df['date'].dt.month / 12)
        df['month_cos'] = np.cos(2 * np.pi * df['date'].dt.month / 12) 
        
    
    if "LST_Day" in df.columns and "LST_Night" in df.columns:
        df['LST_Diff'] = df['LST_Day'] - df['LST_Night']

    # Drop rows with missing target
    df = df.dropna(subset=[target_col]).reset_index(drop=True)
    
    print("Create Database")

    return df


# ---------------------------------------------------
# 3. Preprocessing Definition
# ---------------------------------------------------
def build_preprocessor(X):
    """
    Build ColumnTransformer for numeric + categorical preprocessing.
    - Numeric: median imputation + StandardScaler
    - Categorical: most_frequent imputation + OneHotEncoder
    - station_id is treated as categorical even if numeric
    """
    numeric = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
    numeric = [c for c in numeric if c != "station_id"]

    categorical = X.select_dtypes(include=["object"]).columns.tolist()
    if "station_id" in X.columns and "station_id" not in categorical:
        categorical.append("station_id")

    numeric_transform = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ])

    categorical_transform = Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ])

    preprocessor = ColumnTransformer([
        ("num", numeric_transform, numeric),
        ("cat", categorical_transform, categorical)
    ])

    return preprocessor


# ---------------------------------------------------
# 4. Build Model Pipelines
# ---------------------------------------------------
def build_pipelines(preprocessor):
    """
    Build imbalanced-learn Pipelines with:
    - preprocess
    - SMOTE
    - SelectKBest (mutual_info_classif)
    - classifier
    """
    pipelines = {}

    # SVM
    svm = SVC(
        class_weight="balanced",
        probability=True,
        random_state=RANDOM_STATE
    )
    pipelines["SVM"] = ImbPipeline([
        ("preprocess", preprocessor),
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("select", SelectKBest(mutual_info_classif, k=20)),
        ("clf", svm),
    ])

    # Random Forest
    rf = RandomForestClassifier(
        class_weight="balanced",
        random_state=RANDOM_STATE
    )
    pipelines["RandomForest"] = ImbPipeline([
        ("preprocess", preprocessor),
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("select", SelectKBest(mutual_info_classif, k=20)),
        ("clf", rf),
    ])

    # MLP
    mlp = MLPClassifier(
        max_iter=500,
        random_state=RANDOM_STATE
    )
    pipelines["MLP"] = ImbPipeline([
        ("preprocess", preprocessor),
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("select", SelectKBest(mutual_info_classif, k=20)),
        ("clf", mlp),
    ])

    return pipelines


# ---------------------------------------------------
# 5. Hyperparameter Search Spaces
# ---------------------------------------------------
def param_spaces():
    """
    Hyperparameter spaces for each model, including SelectKBest.k
    """
    return {
        "SVM": {
            "select__k": [10, 15, 20, 25, 30],
            "clf__C": np.logspace(-2, 2, 6),
            "clf__gamma": ["scale", "auto"] + list(np.logspace(-3, 1, 5)),
            "clf__kernel": ["rbf", "poly", "sigmoid"],
        },
        "RandomForest": {
            "select__k": [10, 15, 20, 25, 30],
            "clf__n_estimators": [100, 200, 300],
            "clf__max_depth": [5, 10, 20, None],
            "clf__min_samples_split": [2, 5, 10],
        },
        "MLP": {
            "select__k": [10, 15, 20, 25, 30],
            "clf__hidden_layer_sizes": [(50,), (100,), (50, 50)],
            "clf__alpha": [0.0001, 0.001, 0.01],
            "clf__activation": ["relu", "tanh"],
            "clf__learning_rate": ["constant", "adaptive"],
        },
    }


# ---------------------------------------------------
# 6. Hyperparameter Tuning
# ---------------------------------------------------
def tune_model(name, pipeline, params, X_train, y_train):
    """
    RandomizedSearchCV with F1-weighted scoring.
    """
    print(f"\n🔎 Tuning {name} ...")

    search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=params,
        scoring="f1_weighted",
        cv=3,
        n_iter=20,
        n_jobs=-1,
        random_state=RANDOM_STATE,
        verbose=1,
    )
    search.fit(X_train, y_train)

    print(f"Best params for {name}: {search.best_params_}\n")
    return search.best_estimator_


# ---------------------------------------------------
# 7. Test-set Evaluation (incl. per-class report)
# ---------------------------------------------------
def evaluate_test(model, X_test, y_test, name, outdir):
    """
    Evaluate model on test set:
    - global weighted metrics
    - per-class precision/recall/F1
    - normalized confusion matrix (saved as PNG)
    - saves metrics & classification report as CSV
    """
    preds = model.predict(X_test)

    # Global weighted metrics
    results = {
        "accuracy": accuracy_score(y_test, preds),
        "precision_weighted": precision_score(
            y_test, preds, average="weighted", zero_division=0
        ),
        "recall_weighted": recall_score(
            y_test, preds, average="weighted", zero_division=0
        ),
        "f1_weighted": f1_score(
            y_test, preds, average="weighted", zero_division=0
        ),
    }

    print(f"\n📊 Test Results for {name}:")
    print(pd.DataFrame([results]))

    # Per-class metrics
    report_dict = classification_report(
        y_test, preds, output_dict=True, digits=3, zero_division=0
    )
    report_df = pd.DataFrame(report_dict).T
    report_df.to_csv(os.path.join(outdir, f"{name}_classification_report.csv"))

    # Confusion matrix (normalized by true class)
    cm = confusion_matrix(y_test, preds, normalize="true")
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, cmap="Blues", fmt=".2f")
    plt.title(f"{name} - Normalized Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.tight_layout()
    plt.savefig(os.path.join(outdir, f"{name}_confusion_matrix.png"), dpi=300)
    plt.close()

    # Save global test metrics
    pd.DataFrame([results]).to_csv(
        os.path.join(outdir, f"{name}_test_results.csv"), index=False
    )

    return results


# ---------------------------------------------------
# 8. K-fold Cross-Validation
# ---------------------------------------------------
def evaluate_kfold(model, X, y, name, outdir):
    """
    5-fold Stratified CV with multiple metrics.
    Returns mean scores (as Series).
    """
    print(f"\n🔁 Running 5-fold Stratified CV for {name}...")

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

    scores = cross_validate(
        model,
        X,
        y,
        scoring={
            "accuracy": "accuracy",
            "precision_weighted": "precision_weighted",
            "recall_weighted": "recall_weighted",
            "f1_weighted": "f1_weighted",
        },
        cv=skf,
        n_jobs=-1,
        return_train_score=False,
    )

    df_scores = pd.DataFrame(scores)
    df_scores.to_csv(os.path.join(outdir, f"{name}_kfold_results.csv"), index=False)

    print(f"K-fold mean scores for {name}:")
    print(df_scores.mean())

    return df_scores.mean()


# ---------------------------------------------------
# 9. Feature Importance via Permutation Importance
# ---------------------------------------------------
def compute_feature_importances_across_models(models, X_ref, y_ref, outdir, top_n=15):
    """
    Compute permutation-based feature importances for each model
    using the SAME reference dataset (X_ref, y_ref),
    then save per-model CSVs and a combined comparison CSV.
    """
    comparison = pd.DataFrame(index=X_ref.columns)
    all_model_dfs = {}

    for name, model in models.items():
        print(f"\n⭐ Computing permutation importance for {name} ...")

        r = permutation_importance(
            model,
            X_ref,
            y_ref,
            n_repeats=10,
            random_state=RANDOM_STATE,
            n_jobs=-1,
            scoring="f1_weighted",
        )

        fi_df = pd.DataFrame({
            "feature": X_ref.columns,
            "importance_mean": r.importances_mean,
            "importance_std": r.importances_std,
        }).sort_values("importance_mean", ascending=False)

        # Save full table per model
        fi_df.to_csv(os.path.join(outdir, f"{name}_feature_importances.csv"), index=False)
        all_model_dfs[name] = fi_df

        # Add to comparison matrix (using mean importance)
        comparison[name] = fi_df.set_index("feature")["importance_mean"]

        # Plot top N
        top = fi_df.head(top_n).sort_values("importance_mean", ascending=True)
        plt.figure(figsize=(8, 6))
        plt.barh(top["feature"], top["importance_mean"])
        plt.xlabel("Permutation importance (mean Δ F1_weighted)")
        plt.title(f"Top {top_n} features - {name}")
        plt.tight_layout()
        plt.savefig(
            os.path.join(outdir, f"{name}_feature_importances_top{top_n}.png"),
            dpi=300
        )
        plt.close()

    # Save comparison table (features × models)
    comparison.to_csv(os.path.join(outdir, "feature_importance_comparison.csv"))

    print("\n==============================")
    print("Feature importance comparison - head:")
    print("==============================")
    print(comparison.head(20))

    return comparison, all_model_dfs


# ---------------------------------------------------
# 10. Run pipeline for a single target and save in its own folder
# ---------------------------------------------------
def run_pipeline(db_path, target_col, output_root, selected_stations, selected_columns, start_date, end_date, table_name, drop_cols):
    """
    Run the full pipeline for a given target_col.
    All outputs are stored in: output_root/target_col/
    """
    print(f"\n=== Running pipeline for target: {target_col} ===")
    run_tag = target_col.replace(" ", "_")
    outdir = os.path.join(output_root, run_tag)
    os.makedirs(outdir, exist_ok=True)

    # Load data
    df = load_data_sqlite(db_path=db_path, target_col=target_col, table_name=table_name, selected_stations=selected_stations, selected_columns=selected_columns, start_date=start_date, end_date=end_date)
    feature_cols = [col for col in df.columns if col not in drop_cols + [target_col]]
    X = df.drop(columns=feature_cols)
    y = df[target_col]

    # Class imbalance analysis
    plot_class_distribution(
        y,
        f"Overall class distribution ({target_col})",
        os.path.join(outdir, "classes_overall.png")
    )

    # Train/test split (80/20, stratified)
    X_train, X_test, y_train, y_test = train_test_split(
        X,
        y,
        test_size=0.2,
        stratify=y,
        random_state=RANDOM_STATE,
    )

    plot_class_distribution(
        y_train,
        f"Train class distribution ({target_col})",
        os.path.join(outdir, "classes_train.png")
    )
    plot_class_distribution(
        y_test,
        f"Test class distribution ({target_col})",
        os.path.join(outdir, "classes_test.png")
    )

    # Preprocessing and pipelines
    preprocessor = build_preprocessor(X_train)
    pipelines = build_pipelines(preprocessor)
    params = param_spaces()

    test_results = {}
    kfold_results = {}
    best_models = {}

    # Train, tune, evaluate each model
    for name, pipe in pipelines.items():
        # Hyperparameter tuning
        best_model = tune_model(name, pipe, params[name], X_train, y_train)
        best_models[name] = best_model

        # Save fitted pipeline
        dump(best_model, os.path.join(outdir, f"{name}_pipeline.joblib"))

        # Test-set evaluation
        test_results[name] = evaluate_test(
            best_model, X_test, y_test, name, outdir
        )

        # K-fold CV on full dataset (for robustness)
        kfold_results[name] = evaluate_kfold(
            best_model, X, y, name, outdir
        )

    # Save comparison of test metrics
    test_df = pd.DataFrame(test_results).T
    test_df.to_csv(os.path.join(outdir, "model_comparison_testset.csv"))

    # Save comparison of k-fold metrics
    kfold_df = pd.DataFrame(kfold_results).T
    kfold_df.to_csv(os.path.join(outdir, "model_comparison_kfold.csv"))

    print("\n===========================")
    print(f" FINAL MODEL COMPARISON (Test set) - {target_col}")
    print("===========================")
    print(test_df)

    # Feature importance comparison across models
    compute_feature_importances_across_models(
        best_models,
        X_test,
        y_test,
        outdir,
        top_n=15
    )


# ---------------------------------------------------
# 11. Main: loop over multiple targets
# ---------------------------------------------------
def main():
    
    db_path = "../database/database.db"
    
    selected_stations = {
        "ALL": ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid', 'Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht', 'Ramsar', 'Nowshahr'],
        "Cat_1": ['Babolsar', 'Gharakhil', 'Sari', 'Sari (dasht-e-naz airport)', 'Galugah', 'Bandar-e-amirabad', 'Amol', 'Polsefid'],
        "Cat_2": ['Siahbisheh', 'Kiyasar', 'Kojur', 'Baladeh', 'Alasht'],
        "Cat_3": ['Ramsar', 'Nowshahr'],
    }
    
    selected_columns = [
        'region_id',
        'region_name',
        'station_name',
        'station_id',
        'lat',
        'lon',
        'station_elevation',
        'date',
        'rrr24',
        'Hargreaves',
        'GPM',
        'MOD16A2GF',
        'NDVI',
        'EVI',
        'LST_Day',
        'LST_Night',
        'LST',
        'PCI_GPM',
        'VCI',
        'TCI',
        'TCI_Day',
        'TCI_Night',
        'VHI',
        'VHI_Day',
        'VHI_Night',
        'CI_GPM',
    ]

    targets = [
        "SPEI_1_Class",
        "SPEI_3_Class",
        "SPEI_6_Class",
        "SPEI_9_Class",
        "SPEI_12_Class",
        "SPEI_15_Class",
        "SPEI_18_Class",
        "SPEI_21_Class",
        "SPEI_24_Class",
        "SPI_1_Class",
        "SPI_3_Class",
        "SPI_6_Class",
        "SPI_9_Class",
        "SPI_12_Class",
        "SPI_15_Class",
        "SPI_18_Class",
        "SPI_21_Class",
        "SPI_24_Class",
    ]
    
    for name, cat in selected_stations.items():
        for t in targets:
            print(f"Cat: {name} - Target: {t}")
            drop_cols = ['region_id', 'region_name', 'station_name', 'station_id', 'date', 'rrr24', 'Hargreaves', f'GPM_{t}']
            run_pipeline(
                db_path=db_path,
                target_col=t,
                output_root=f"../output/results/{name}",
                selected_stations=cat,
                selected_columns = selected_columns + ["_".join(t.split("_")[:2]), f"GPM_{"_".join(t.split("_")[:2])}"],
                start_date="2006-09",
                end_date="2025-09",
                table_name="data",
                drop_cols=drop_cols
            )


if __name__ == "__main__":
    main()


KeyboardInterrupt: 